In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Bachelorarbeit/Dataset_Vision/eval_dataset_food

/content/drive/MyDrive/Bachelorarbeit/Dataset_Vision/eval_dataset_food


In [ ]:
pip install -U transformers timm torchmetrics

# DETR

In [ ]:
from transformers import DetrFeatureExtractor, DetrForObjectDetection
import torch
from PIL import Image




In [ ]:
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-101")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101")

In [ ]:
img_path = "/content/drive/MyDrive/Bachelorarbeit/Dataset_Vision/eval_dataset_food/5.jpg"
image = Image.open(img_path)

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)

target_sizes = torch.tensor([image.size[::-1]])
results = feature_extractor.post_process(outputs, target_sizes=target_sizes)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    if score > 0.9:
        print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
        )

Detected dining table with confidence 0.955 at location [-0.08, -0.09, 199.92, 199.91]
Detected cup with confidence 0.999 at location [-0.05, 7.68, 68.54, 86.49]
Detected fork with confidence 0.943 at location [178.94, 61.61, 200.0, 114.71]


# Evaluation

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
"""
ds_collection = {}

for i in range(5):
  ds_collection.update({f"ds{i+1}": pd.read_csv(f"/content/drive/MyDrive/Bachelorarbeit/Dataset_Vision/eval_dataset_food/{i+1}.csv")})"""

ds1 = pd.read_csv("/content/drive/MyDrive/Bachelorarbeit/Dataset_Vision/eval_dataset_food/5.csv")

In [ ]:
"""for obj in ds_collection.keys():
  for i in range(len(ds_collection[obj])):
    bbox = torch.tensor([[ds_collection[obj]["X_min"][i],ds_collection[obj]["Y_min"][i],ds_collection[obj]["Width"][i]+ds_collection[obj]["X_min"][i],ds_collection[obj]["Height"][i]+ds_collection[obj]["Y_min"][i]]], dtype=torch.float)
    cls = ds_collection[obj]["Object"][i]
    target_dict.update({cls:bbox})

pprint(target_dict)"""

target_dict = {}
for i in range(len(ds1)):
    bbox = torch.tensor([[ds1["X_min"][i],ds1["Y_min"][i],ds1["Width"][i]+ds1["X_min"][i],ds1["Height"][i]+ds1["Y_min"][i]]], dtype=torch.float)
    cls = ds1["Object"][i]
    target_dict.update({cls:bbox})

pred_dict = {}

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = torch.tensor([[round(i, 2) for i in box.tolist()]])
    
    if score > 0.9:
     pred_dict.update({model.config.id2label[label.item()]:box})




      

## IoU (Intersection over Union)

In [ ]:
# Object Detection Metrics
# IoU, AP (average precision) mAP (mean average precision)

import torchvision.ops.boxes as bops

iou_values = []

for obj in target_dict.keys():
  traget = target_dict[obj]
  prediction = torch.round(pred_dict[obj])
  iou = bops.box_iou(traget, prediction)
  iou_values.append(iou.item())
print(np.mean(np.array(iou_values)))

0.9295800924301147
